In [1]:
!pip install pandas_datareader yfinance

In [2]:
import pandas as pd
import requests
from pandas_datareader import data as pdr
import yfinance as yf
import math

In [3]:
source_file = "data.xlsx" # исходник данных для получения universe
start = '2017-01-01' # дата начало получения для finance.yahoo.com
end = '2021-09-04' # дата окончания получения для finance.yahoo.com
dates_for_moex = ['2020-09-03', '2021-09-03']


In [4]:
main_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "universe")

naufor_df = main_data[main_data['source']=="naufor.ru"]
naufor_codes = naufor_df['code'].to_numpy().tolist()

yfinance_df = main_data[main_data['source']=="finance.yahoo.com"]
yfinance_codes = yfinance_df['code'].to_numpy().tolist()


# Функции экстракции

In [5]:
def get_pif_naufor_quotes(code, start_date, end_date):
    '''
    получаем данные с naufor.ru
    '''
    start_date = start_date.replace("-", '')
    end_date = end_date.replace("-", '')
    referer = 'http://pif.naufor.ru/pif.asp?act=view&id={}'.format(code)
    url = 'http://pif.naufor.ru/pif.asp?act=getpayschajson&id={}&begindate={}&enddate={}'.format(code, start_date, end_date)
    resp = requests.get(url, headers={'referer': referer})
    
    blank_array = []
    result_dict = resp.json()
    
    for index, date in enumerate(result_dict['Label']): 
        dd = date.split('.')
        blank_array.append({
            'source': 'naufor.ru',
            'code': code,
            'date': "{year}-{month}-{day}".format(year=dd[2], month=dd[1], day=dd[0]),
            'price': result_dict['UnitValue'][index]
        })

    return blank_array

def get_yahoo_finance_quote(ticker, Start, End):
    '''
    получаем данные с finance.yahoo.com
    '''
    yf.pdr_override() 
    data2 = pdr.get_data_yahoo(ticker, start=Start, end=End)
    output = pd.DataFrame(data=data2['Close'])
    output['code'] = ticker
    clr = output.reset_index().to_dict('records')
    res = []
    for row in clr:
        res.append({
            'source': 'finance.yahoo.com',
            'code': ticker,
            'date': row['Date'].strftime('%Y-%m-%d'),
            'price': row['Close']
        })
    return res

def get_securities(date):
    page_size = 100
    all_rows = []
    start = 0
    url_temp = "https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?date={date}&start={start}&iss.json=extended"
    r = requests.get(url_temp.format(start=start, date=date))
    res = r.json()
    all_rows += res[1]['history'][1]
    total = res[1]['history.cursor'][1][0]['TOTAL']
    pages = math.ceil(total/page_size)+1
    print("Date:{} Total:{}, Pages:{}".format(date, total, pages))
    if total <=100:
        return all_rows
    for page in range(1, pages):
        start = page*page_size
        r = requests.get(url_temp.format(start=start, date=date))
        res = r.json()
        all_rows += res[1]['history'][1]
    return all_rows


# Извлечение данных с naufor

In [6]:
all_data = []
for code in naufor_codes:
    all_data += get_pif_naufor_quotes(code, start, end)
    print("[naufor]Ready code: {}".format(code))

[naufor]Ready code: 299
[naufor]Ready code: 60
[naufor]Ready code: 2746
[naufor]Ready code: 4017
[naufor]Ready code: 2880
[naufor]Ready code: 651
[naufor]Ready code: 3830
[naufor]Ready code: 662
[naufor]Ready code: 525
[naufor]Ready code: 63
[naufor]Ready code: 2322
[naufor]Ready code: 3823
[naufor]Ready code: 4394
[naufor]Ready code: 66
[naufor]Ready code: 606
[naufor]Ready code: 256


In [7]:
for code in yfinance_codes:
    all_data += get_yahoo_finance_quote(code, start, end)
    print("[get_yahoo_finance_quote]Ready code: {}".format(code))

[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GAZP.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: GMKN.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: LKOH.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MGNT.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: MTSS.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NLMK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: NVTK.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_quote]Ready code: PLZL.ME
[*********************100%***********************]  1 of 1 completed
[get_yahoo_finance_

In [8]:
df = pd.DataFrame(all_data)
df.to_excel("tickers_prices.xlsx", index=False)

# извлечение с moex

In [9]:
secs = []
for date in dates_for_moex:
    secs += get_securities(date)
len(secs)

Date:2020-09-03 Total:2059, Pages:22
Date:2021-09-03 Total:2376, Pages:25


4435

In [10]:
bonds_df = pd.DataFrame(secs)
bonds_prices = bonds_df[['SHORTNAME','BOARDID','TRADEDATE','SECID','LEGALCLOSEPRICE','MATDATE','DURATION','YIELDATWAP', 'COUPONPERCENT','FACEUNIT','FACEVALUE']]

#готовим списко облигаций
asset_class_list = ["ОФЗ", 'Облигации корпоративные', 'Облигации высокодоходные']
bonds_series = main_data.asset_class.isin(asset_class_list)
bonds = main_data[bonds_series]['code']
bonds_list = bonds.values.tolist()
bonds_list #получили список бондов

['RU000A102SV8',
 'RU000A103C46',
 'RU000A101QN1',
 'RU000A100Z91',
 'RU000A0JXQK2',
 'RU000A0ZYT40',
 'RU000A100VR4',
 'RU000A101Z74',
 'RU000A1002E8',
 'RU000A103G42',
 'RU000A103661',
 'RU000A1025U5',
 'RU000A103G75',
 'RU000A0ZYG52',
 'RU000A101QW2',
 'RU000A0ZYFC6',
 'RU000A0JUV08',
 'SU26229RMFS3',
 'SU26234RMFS3',
 'RU000A100K80',
 'SU26223RMFS6',
 'SU26222RMFS8',
 'SU26227RMFS7',
 'SU25084RMFS3',
 'SU26215RMFS2',
 'SU26226RMFS9',
 'SU26219RMFS4',
 'SU26207RMFS9',
 'SU26211RMFS1',
 'SU26220RMFS2',
 'SU26209RMFS5',
 'RU000A1005T9',
 'RU000A103JR3',
 'RU000A102KG6',
 'RU000A103M36',
 'RU000A0JWN63']

In [11]:
#фильтруем датасет с ценами по списку isin из нашего юниверса
filtered_bonds_prices = bonds_prices[bonds_prices.SECID.isin(bonds_list)]

#filtered_bonds_prices = filtered_bonds_prices[filtered_bonds_prices['BOARDID']=='TQCB'] #оставляем только котировки из стаканов
filtered_bonds_prices.sort_values(by=['SECID', 'TRADEDATE'])

filtered_bonds_prices.to_excel("filtered_bonds_prices.xlsx", index=False)



In [12]:
filtered_bonds_prices


,SHORTNAME,BOARDID,TRADEDATE,SECID,LEGALCLOSEPRICE,MATDATE,DURATION,YIELDATWAP,COUPONPERCENT,FACEUNIT,FACEVALUE
237,ГПБ БО-10,TQCB,2020-09-03,RU000A0JUV08,100.080,2024-09-26,23.0,6.55,6.89,RUB,1000.0
496,МОСТРЕСТ 7,TQCB,2020-09-03,RU000A0JWN63,103.450,2026-07-07,631.0,7.55,9.25,RUB,1000.0
632,Роснфт1P4,TQCB,2020-09-03,RU000A0JXQK2,107.500,2027-04-22,864.0,5.64,8.65,RUB,1000.0
727,МТС 001P-3,TQCB,2020-09-03,RU000A0ZYFC6,104.720,2022-11-03,728.0,5.43,7.70,RUB,1000.0
733,Ростел1P3R,TQCB,2020-09-03,RU000A0ZYG52,104.250,2027-11-09,740.0,5.66,7.70,RUB,1000.0
...,...,...,...,...,...,...,...,...,...,...,...
4143,ОФЗ 26223,TQOB,2021-09-03,SU26223RMFS6,99.420,2024-02-28,852.0,6.87,6.50,RUB,1000.0
4146,ОФЗ 26226,TQOB,2021-09-03,SU26226RMFS9,105.050,2026-10-07,1529.0,6.87,7.95,RUB,1000.0
4147,ОФЗ 26227,TQOB,2021-09-03,SU26227RMFS7,101.695,2024-07-17,956.0,6.86,7.40,RUB,1000.0
4149,ОФЗ 26229,TQOB,2021-09-03,SU26229RMFS3,101.326,2025-11-12,1324.0,6.88,7.15,RUB,1000.0
